## Scraper for r/jokes subreddit that will be used to train NLP model

In [1]:
from collections import defaultdict
import datetime
from datetime import datetime
import pandas as pd
import praw
import time

In [176]:
# Test request of jokes subreddit, hot posts, limited
import r_auth

reddit = praw.Reddit(client_id = r_auth.client_id,
                     client_secret = r_auth.client_secret,
                     user_agent = r_auth.user_agent,
                     username = r_auth.username,
                     password = r_auth.password)



# Set parameters for Jokes subreddit query
submissions = reddit.subreddit('Jokes').top(limit = 100)


# Set timeframe for joke scraping to one month
TODAY = date.today()

timestamp = time.mktime(TODAY.timetuple())
seconds_one_month = 2_628_288 * 100
one_month_timeframe = timestamp - seconds_one_month


# Create default dictionary, and then set the keys in the for loop for each required field
joke_dict = defaultdict(list)

for joke in submissions:
    if joke.created > one_month_timeframe and joke.score > 2_000:
        joke_dict['title'].append(joke.title)
        joke_dict['body'].append(joke.selftext)
        joke_dict['score'].append(joke.score)
        joke_dict['date'].append(datetime.utcfromtimestamp(joke.created).strftime('%Y-%m-%d'))

        
df = pd.DataFrame(joke_dict)

df

,title,body,score,date
0,Sad News: The founder of /r/jokes has passed away,"RIP Larry Tesler, the UI designer that created...",142732,2020-02-20
1,Calm down about the Net Neutrality thing...,Paying additional money to access certain site...,136349,2017-11-22
2,V,V\n\n*Edit: seems like the ctrl key on my keyb...,106421,2017-07-06
3,If your surprised that Jeffrey Epstein commite...,Imagine how surprised he must have been.\n\nEd...,103659,2019-08-10
4,A new Navy recruit has his first day on the su...,"He speaks with the officer, who assigns him hi...",98261,2018-10-08
...,...,...,...,...
95,CAN ADMINS OF THIS SUBREDDIT REDDIT DO A BETTE...,"WE HAVE A NEW MEMBER, A WOMAN. SHE’S BEEN PRIV...",52783,2020-09-21
96,Looks like Trump is keeping up Michelle's idea...,One day in office and he has thousands of peop...,52769,2017-01-22
97,If Queen Elizabeth accidentally farts during d...,Noble gases should have no reaction.,52754,2018-08-28
98,*Nsfw* The military is cutting staff and decid...,"All of them are old, grizzled men who had seen...",52703,2020-11-09


In [177]:
# Create records of jokes in AWS DynamoDB

import boto3
import json

